# 4 point dwell time accuracy/precision measurements. 

A script that determines the accuracy of a four-point measurement by finding the mean of the normalised dataset and comparing this to the fluorescence intensity values from the full CW ODMR curve. 

In [11]:
import numpy as np 
import os 
import pandas as pd

In [2]:
def get_data(file_name):
    
    data = np.loadtxt(file_name)
    return(data)

In [5]:
def load_params(file_path):
    # load in the params data. 
    params_data = pd.read_table(file_path, names = ['Parameter name', 'Parameter value']) 
    return(params_data)

In [35]:
def accuracy_measurement(cw_intensity, four_point_intensity):
    '''Finds the root mean square difference between the CW intenisty and the four-point measurement.'''
    rms_error = []
    for i in range(len(cw_intensity)):
        rms_error.append(np.sqrt(np.power(cw_intensity[i] - four_point_intensity[i],2)))
    return(rms_error)

In [42]:
def reshape_data(reference_data, odmr_data, params_values):
    '''Here we re-shape the reference and the ODMR datasets into a 3D array, 
    based on the number of line scans, frequnecy points and repeats per frequnecy 
    have been used to build the ODMR dataset.'''
    
    # Create variables with the frequnecy scan parameters. 
    num_of_line_scans = int(params_values[4])
    num_of_freqs =  int(1000*(params_values[1]-params_values[0])/params_values[2]) + 1
    points_per_freq = int(params_values[8])
    #print(num_of_line_scans, num_of_freqs, points_per_freq)
    # Reshape the ODMR and reference datasets. 

    odmr_shape_3D = odmr_data[0:num_of_freqs*points_per_freq].reshape(num_of_line_scans, num_of_freqs, points_per_freq)
    reference_data_3D = reference_data[0:num_of_freqs*points_per_freq].reshape(num_of_line_scans, num_of_freqs, points_per_freq)
    
    return(odmr_shape_3D, reference_data_3D)

In [43]:
def normalise_data(reference_data, odmr_data):
    '''Here we normalise the ODMR dataset to the reference data.'''

    norm_odmr_data = odmr_data/reference_data
    return(norm_odmr_data)

In [44]:
def mean_data(odmr_data):
    # find the mean data 
    mean_odmr_data_points =  np.mean(odmr_data, axis = 2)
    mean_odmr_data_points_and_linescans =  np.mean(mean_odmr_data_points, axis = 0)
    
    return(mean_odmr_data_points, mean_odmr_data_points_and_linescans)

In [69]:
def save_data(file_name, dwell_time, repeats, mean_ref_intensity, std_ref_intensity, mean_norm_odmr_full,
         std_odmr_data, accuracy): 
    '''save the data into a text file on my H drive.'''
    
    save_folder = 'H:\\My Documents\\Strathclyde_Back_up\\Thesis and Planning\\Thesis_plots\\' + date + '_2\\'
    # Checking to see if the folder exists in which to save the data, and creating it if it does not
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)
        # Checking the output file exists and creating and initialising it if not
    
    file = save_folder + date  + '_Dwell_time_results_I_'   
    
    for i in range(4):
        output_data_file = file + str(i+1) + '.dat'
        if os.path.isfile(output_data_file) == False:
            with open(output_data_file, "w") as wf:
                wf.write('File name'  + '\t' + 'Number of Repeats' + '\t' + 
                         'Dwell time (ms)' + '\t' + 'Mean counts per single point (N)' + 
                         '\t' + 'std(N)' + '\t' +'Mean Normalised Counts (n)' + 
                         '\t' + 'Standard Deviation of n' + 
                         '\t' + 'Standard Error of n' + '\t' + 'Accuracy of measurement |I_CW - I_4point|' + '\n')
        
        
        with open(output_data_file, "a") as wf:
            wf.write(date + '_' + file_name  + '\t' +
                 str(repeats) + "\t"  + str(dwell_time) + "\t" + 
                 str(mean_ref_intensity[i])  + "\t" + str(std_ref_intensity[i]) + 
                "\t" +  str(mean_norm_odmr_full[i]) + "\t"  + str(std_odmr_data[i]) + "\t" + 
                 str(std_odmr_data[i]/np.sqrt(repeats)) + '\t' +str(accuracy[i]) +'\n')       
        
    # Writing contrast dip description to the .dat file
    

### Start the code analysis

In [66]:
# set the date and the folder location for the dataset 

date = '20210921'
folder_loc = 'I:\\Science\\Physics-Nanobiophotonics\\Group\\User Data\\ODMR\\ODMR and imaging\\2021\\' + date + '\\'

cw_intensities = np.array([0.96696511, 0.95464444, 0.95375027, 0.96396645])

In [123]:
# set file name 
file_name = '20210921_4_point_referenced_odmr_49'

odmr_data = get_data(folder_loc + file_name + '_ODMR1.dat')
reference_data = get_data(folder_loc + file_name + '_REF1.dat')
params_file= load_params(folder_loc + file_name + '_Params.txt')
params_values = params_file['Parameter value']

dwell_time = params_values[3]/1000
repeats = params_values[8]
# reshape the data into a nice format. 
odmr_data, reference_data = reshape_data(reference_data, odmr_data, params_values)

 # Normalise the ODMR data against the reference data. 
normalised_ODMR = normalise_data(reference_data, odmr_data)

# Mean of the normalise dataset. 
# mean_odmr_freq_points = mean for all the points recorded per frequnecy, but not linescans. 
# mean_norm_odmr_full = Mean fluorescence value for all ODMR data points taken per freq. 
mean_odmr_freq_points, mean_norm_odmr_full_disordered = mean_data(normalised_ODMR)
mean_norm_odmr_full = np.roll(mean_norm_odmr_full_disordered, 1)

accuracy = accuracy_measurement(cw_intensities, mean_norm_odmr_full)
std_odmr_data = np.std(normalised_ODMR, axis = 2)

mean_ref_intensity = np.mean(reference_data, axis = 2)
std_ref_intensity = np.std(reference_data, axis = 2)

save_data(file_name, dwell_time, repeats, mean_ref_intensity[0], std_ref_intensity[0], mean_norm_odmr_full,
         std_odmr_data[0], accuracy)

In [78]:
#save_data(file_name, dwell_time, repeats, mean_ref_intensity[0], std_ref_intensity[0], mean_norm_odmr_full,
#         std_odmr_data[0], accuracy)

In [55]:
print(mean_ref_intensity)
print(std_ref_intensity)
print(mean_norm_odmr_full)
print(std_odmr_data)
print(accuracy)

[[980.   990.23 977.9  972.79]]
[[60.960479   57.32344285 56.6101581  55.86435268]]
[0.96039144 0.96409588 0.94210431 0.95878488]
[[0.08516241 0.08754808 0.08390335 0.07185605]]
[0.006573674926617423, 0.009451441326185361, 0.011645956767228238, 0.005181571340644009]
